# GRU

**Gated Recurrent Unit (GRU)** is a type of [Recurrent Neural Network (RNN)](https://fedmug.github.io/kbtu-ml-book/rnn/vanilla_rnn.html) architecture that was introduced in 2014 by Cho et al. It is similar to [Long Short-Term Memory (LSTM)](https://fedmug.github.io/kbtu-ml-book/rnn/lstm.html) in that it can capture long-term dependencies in sequential data, but it has fewer parameters and is faster to train.